In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv, find_dotenv
import unidecode
from lyricsgenius import Genius
from difflib import SequenceMatcher
from joblib import Parallel, delayed
import time

In [2]:
env_path = find_dotenv()
load_dotenv(env_path)

True

In [3]:
# loading the dataset into a dataframe
data = pd.read_csv('../raw_data/data.csv')

# create a sample to work with a smaller dataset throughout development
data_sample = data[50001:55000]
# Fede [:50000]
# Yann [50001:100000]
# Adri [100001:]

# get credentials for Genius API
genius_token = os.environ.get('LYRICSGENIUS_ACCESS_TOKEN')

# instantiate the Genius class with some useful hyperparameters
genius = Genius(genius_token, 
                timeout=220, 
                remove_section_headers=True,
                skip_non_songs=True)

In [4]:
data.loc[170652]

valence                                       0.642
year                                           2020
acousticness                                  0.132
artists                       ['KEVVO', 'J Balvin']
danceability                                  0.856
duration_ms                                  189507
energy                                        0.721
explicit                                          1
id                           7HmnJHfs0BkFzX4x8j0hkl
instrumentalness                            0.00471
key                                               7
liveness                                      0.182
loudness                                     -4.928
mode                                              1
name                Billetes Azules (with J Balvin)
popularity                                       74
release_date                             2020-10-16
speechiness                                   0.108
tempo                                        94.991
Name: 170652

In [5]:
# clean strings
def clean_text(text):
    """
    Removes symbols, accents and uppercases from text.
    """
    text = text.replace("[",'')\
        .replace("]",'')\
        .replace("'",'')\
        .replace(",",'')\
        .replace(":",'')\
        .replace(")",'')\
        .replace("(",'')\
        .replace(".",'')\
        .replace('"','')\
        .replace('/','')\
        .replace("\\",'')\
        .replace("(?)",'')\
        .replace("-",'')
        
    return unidecode.unidecode(text.lower())

In [6]:
# get lyrics
def get_lyrics(artist, song_title, genius):
    """
    Returns lyrics from Genius API.
    """
    # cleanup the inputs
    artist = clean_text(artist)
    song_title = clean_text(song_title)
    
    # call the API to search for the song
    song = genius.search_song(title=song_title, artist=artist)

    if song != None:
        # get song name from the API response to compare similarity
        api_response = unidecode.unidecode(song.to_dict()['full_title']\
            .replace('\xa0', ' ')\
            .replace('\u200b', ' ')\
            .lower())
        api_response = api_response.split(' by ')
        # check similarity
        song_similar = similar(api_response[0], song_title)
        # artist_similar = similar(api_response[1], artist)

        if song_similar >= 0.9:
            # all lyrics responses come with the song's title and 'Lyrics' str
            # so we count how many characters should be removed in order 
            # to delete any extra text
            characters_to_remove = len(song.to_dict()['title'] + ' Lyrics')   
            
            # get lyrics from API
            lyrics = genius.lyrics(song.to_dict()['id'])[characters_to_remove:-5]\
                .replace('\n', ' ')\
                .replace('\u205f', ' ')\
                .replace('\u2005', ' ')\
                .replace('\\', ' ')\
                .strip()

        else:
            lyrics = 'None'
    
    else:
        lyrics = 'None'
    
    return lyrics


# we need to check similarity between the input of the API call and the response
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [7]:
# create df containing lyrics, sentiments and emotions
def get_lyrics_df(df, genius):
    """
    Returns a DataFrame adding lyrics, sentiments and emotions.
    """
    full_songs = Parallel(n_jobs=os.cpu_count())\
        (delayed(get_lyrics)(artist, song_title, genius)\
        for artist, song_title in zip(df['artists'], df['name']))
    
    df['lyrics'] = full_songs
    
    return df

In [25]:
start = time.time()
data_lyrics = get_lyrics_df(data_sample, genius)
end = time.time()
print(f"get_lyrics_df for {len(data_sample)} samples: {(end - start)/60} m")

KeyboardInterrupt: 

In [8]:
amount = 1000
max = 170653 - 1
start = 128001
end = start + amount
iterations = int(((max+1) - 50000)/amount)

start_time = time.time()
for i in range(iterations):
    if start <= max:
        start_time_loop = time.time()
        data_lyrics = get_lyrics_df(data[start:end], genius)
        data_lyrics.to_csv('../raw_data/data_lyrics_B.csv', mode='a', header=False)
        end_time_loop = time.time()
        print(f'from {start} to {end-1} saved ✅')
        start += amount
        end = start + amount
        print(f'elapsed time: {(end_time_loop - start_time_loop)/60} m')
        time.sleep(5)
    else:
        print('done')
end_time = time.time()
print(f"get_lyrics_df for {iterations * 5000} samples: {(end_time - start_time)/3600} h")

/var/folders/7k/g6_9j05d4qg33cskxqfft6dh0000gn/T/ipykernel_1077/2896785801.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lyrics'] = full_songs


from 119001 to 120000 saved ✅
elapsed time: 11.761550871531169 m
from 120001 to 121000 saved ✅
elapsed time: 11.379757769902547 m
from 121001 to 122000 saved ✅
elapsed time: 11.799497767289479 m
from 122001 to 123000 saved ✅
elapsed time: 12.04932025273641 m
from 123001 to 124000 saved ✅
elapsed time: 12.451234964529673 m
from 124001 to 125000 saved ✅
elapsed time: 12.638124827543894 m
from 125001 to 126000 saved ✅
elapsed time: 3.213462535540263 m
from 126001 to 127000 saved ✅
elapsed time: 3.1320181528727216 m
from 127001 to 128000 saved ✅
elapsed time: 4.0018212834994 m


KeyboardInterrupt: 

In [ ]:
# save dataframe to csv
data_lyrics.to_csv('../raw_data/data_lyrics_B.csv', mode='a')

In [9]:
# def get_translation(lyric):
#    translator = Translator()
#    lyrics_trans = translator.translate(lyric, dest='en').text
    
#   return lyrics_trans

In [10]:
# create df containing lyrics, sentiments and emotions
#def get_translations_df(df):
#    """
#    Returns a DataFrame adding lyrics, sentiments and emotions.
#    """
#    
#    full_songs = Parallel(n_jobs=os.cpu_count())\
#        (delayed(get_translation)(lyric)\
#        for lyric in df['lyrics'])
#
#    df['lyrics_trans'] = full_songs
#    
#    return df

In [11]:
#start = time.time()
#data_translated_10k = get_translations_df(data_lyrics_10k)
#end = time.time()
#print(f"get_translations_df for {len(data_sample_10k)} samples: {(end - start)/60} m")

In [12]:
# save dataframe to csv
#data_translated_10k.to_csv('../processed_data/data_translated_10k.csv')